Weil für die Analyse bez. der Abhängigkeit von (linken) Mehrheiten Kooperation über den linken Block (SP,GP,AL) womöglich interessanter sind, konstruieren wir hier doch einen entsprechenden Dataframe

In [108]:
#Hier definieren wir die drei Blöcke
Block_Links = ["SP", "Grüne", "AL", "FraP", "CSP"]
Block_Mitte = ["CVP", "GLP", "EVP", "LdU", "Parteilos"]
Block_Rechts = ["FDP", "SVP", "PFZ", "SL"]

In [109]:
import os
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
import re

files = os.listdir("alle_geschaefte")
df_list = []

for file in tqdm(files): 
    
    if "geschaeft" in file: #um fehlerhafte files auszuschliessen
        geschaeft = open("alle_geschaefte/"+ file, "r", encoding = "utf-8")
        text = geschaeft.read()
        soup = BeautifulSoup(text, "html.parser")
    
        #Jetzt filtern wir nur jene raus, die mehrere Einreichende haben:
        einreichende = soup.find_all("div")[24].find_all("a")
        if len(einreichende) > 1: 
            
            #Jetzt filtern wir jene, die überhaupt einen Namen drin haben; Erkennungsmerkmal: Klammern
            einreichender1 = einreichende[0].text.strip()
            if "(" and ")" in einreichender1:
                
                name1 = einreichender1.split(" (")[0]
                partei1 = einreichender1.split(" (")[1][:-1]
                
                einreichender2 = einreichende[1].text.strip()
                name2 = einreichender2.split(" (")[0]
                partei2 = einreichender2.split(" (")[1][:-1]
                
                geschaeft_nr = soup.find_all("div")[22].find_all("span")[1].text
                
                regex = r"((?<=\svom\s)\d*\.\d*\.\s*\d*)"
                datum = re.findall(regex, str(soup))[0]
                
                try: 
                    regex_dep = r"((?<=Zuständiges Departement).*\n*.*)"
                    dep_fragment = re.findall(regex_dep, str(soup))[0]
                    soup_dep = BeautifulSoup(dep_fragment, "html.parser")
                    departement = soup_dep.text[1:]

                except: 
                    departement = ""
                            
                #Zum Schluss: Wir wollen nur jene in unserem Dict mit unterschiedlichen Blocks
                if partei1 in Block_Links:
                    block1 = "links"
                elif partei1 in Block_Mitte:
                    block1 = "mitte"
                else:
                    block1 = "rechts"
                    
                if partei2 in Block_Links:
                    block2 = "links"
                elif partei2 in Block_Mitte:
                    block2 = "mitte"
                else:
                    block2 = "rechts"
                
                if block1 == "links" and block2 != "links":
                    mini_dict = {
                        "Datum" : datum,
                        "Geschäft" : geschaeft_nr,
                        "Einreichender 1" : name1,
                        "Partei 1" : partei1,
                        "Block 1" : block1,
                        "Einreichender 2" : name2,
                        "Partei 2" : partei2,
                        "Block 2" : block2,
                        "Departement" : departement
                    }
                    df_list.append(mini_dict)
                    
                elif block2 == "links" and block1 != "links":
                    mini_dict = {
                        "Datum" : datum,
                        "Geschäft" : geschaeft_nr,
                        "Einreichender 1" : name1,
                        "Partei 1" : partei1,
                        "Block 1" : block1,
                        "Einreichender 2" : name2,
                        "Partei 2" : partei2,
                        "Block 2" : block2,
                        "Departement" : departement
                    }
                    df_list.append(mini_dict)  

100%|██████████| 7026/7026 [05:08<00:00, 22.77it/s]


In [110]:
#Jetzt ist die Liste nicht mehr so lang...
len(df_list)

397

In [111]:
df = pd.DataFrame(df_list)
df.head(5)

,Datum,Geschäft,Einreichender 1,Partei 1,Block 1,Einreichender 2,Partei 2,Block 2,Departement
0,28.09.2011,2011/362,Alecs Recher,AL,links,Maleica Landolt,GLP,mitte,Finanzdepartement (FD)
1,20.10.2004,2004/541,Niklaus Scherr,AL,links,Kurt Maeder,CVP,mitte,Hochbaudepartement (HBD)
2,16.05.2013,2005/550,Mario Mariani,CVP,mitte,Muriel Herzig,Grüne,links,Tiefbau- und Entsorgungsdepartement (TED)
3,06.01.2016,2016/16,Christina Schiller,AL,links,Mario Babini,Parteilos,mitte,Polizeidepartement (PD)
4,1.12.2004,2004/637,Corine Mauch,SP,links,Albert Leiser,FDP,rechts,Finanzdepartement (FD)


In [112]:
#Hats hier noch zusätzliche Leute ohne Parteiangabe, so dass wir das Reparatur-Dict ergänzen müssen? Nein!
df[df["Partei 2"] == ""]["Einreichender 2"].value_counts()

Gregor Bucher          5
Andreas Ammann         4
Placidus Maissen       1
Anton Stäbler          1
Fakir Atalay           1
Urs Lauffer            1
Andreas J. Schlegel    1
Name: Einreichender 2, dtype: int64

In [113]:
Reparatur_dict = {"Gregor Bucher":"Grüne",
                  "Andreas Ammann":"SP",
                  "Urs Rechsteiner":"CVP",
                  "Andreas J. Schlegel":"FDP",
                  "Jürg Liebermann":"FDP",
                  "Peter Marti":"FDP",
                  "Fakir Atalay":"SP",
                  "Anton Stäbler":"CVP",
                  "Placidus Maissen":"CVP",
                  "Salomon Browar":"Grüne",
                  "Monika Piesbergen":"FDP",
                  "Hanna Lienhard":"FDP",
                  "Urs Lauffer":"FDP",
                  "Esther Ponti-Weder":"CVP"}

In [114]:
def Reparatur(Person):
    return Reparatur_dict[Person]

fehlerposition1 = df[df["Partei 1"] == ""]["Einreichender 1"].index.tolist()
fehlerposition2 = df[df["Partei 2"] == ""]["Einreichender 2"].index.tolist()

for zeile in fehlerposition1:
    df.loc[zeile, 'Partei 1'] = df.loc[zeile, 'Partei 1'].replace("", Reparatur(df.loc[zeile, "Einreichender 1"]))

for zeile in fehlerposition2:
    df.loc[zeile, 'Partei 2'] = df.loc[zeile, 'Partei 2'].replace("", Reparatur(df.loc[zeile, "Einreichender 2"]))

Achtung, durch die Reparatur sind möglicherweise Paare des gleichen Blocks entstanden, die müssen wir rausfiltern:

In [115]:
len(df)

397

In [116]:
#Und hier ist die Variante für Blöcke:
#Hier definieren wir die drei Blöcke
Block_Links = ["SP", "Grüne", "AL", "FraP", "CSP"]
Block_Mitte = ["CVP", "GLP", "EVP", "LdU", "Parteilos"]
Block_Rechts = ["FDP", "SVP", "PFZ", "SL"]

for zeile in range(0, len(df)-1):
    
    #erst müssen wir die fehlende Wert für die Blöcke ergänzen...
    if df.loc[zeile, "Partei 1"] in Block_Links:
        df.loc[zeile, "Block 1"] = "links"
    elif df.loc[zeile, "Partei 1"] in Block_Mitte:
        df.loc[zeile, "Block 1"] = "mitte"
    else:
        df.loc[zeile, "Block 1"] = "rechts"
                    
    if df.loc[zeile, "Partei 2"] in Block_Links:
        df.loc[zeile, "Block 2"] = "links"
    elif df.loc[zeile, "Partei 2"] in Block_Mitte:
        df.loc[zeile, "Block 2"] = "mitte"
    else:
        df.loc[zeile, "Block 2"] = "rechts"
    
    #...und dann löschen wir gleiche raus
    if df.loc[zeile, "Block 1"] == df.loc[zeile, "Block 2"]:
        #print( df.loc[zeile, "Block 1"] + df.loc[zeile, "Block 2"])
        df.drop(zeile, inplace = True)

In [117]:
len(df)

360

In [118]:
df

,Datum,Geschäft,Einreichender 1,Partei 1,Block 1,Einreichender 2,Partei 2,Block 2,Departement
0,28.09.2011,2011/362,Alecs Recher,AL,links,Maleica Landolt,GLP,mitte,Finanzdepartement (FD)
1,20.10.2004,2004/541,Niklaus Scherr,AL,links,Kurt Maeder,CVP,mitte,Hochbaudepartement (HBD)
2,16.05.2013,2005/550,Mario Mariani,CVP,mitte,Muriel Herzig,Grüne,links,Tiefbau- und Entsorgungsdepartement (TED)
3,06.01.2016,2016/16,Christina Schiller,AL,links,Mario Babini,Parteilos,mitte,Polizeidepartement (PD)
4,1.12.2004,2004/637,Corine Mauch,SP,links,Albert Leiser,FDP,rechts,Finanzdepartement (FD)
...,...,...,...,...,...,...,...,...,...
392,28.08.2013,2013/295,Simon Kälin-Werth,Grüne,links,Martin Luchsinger,GLP,mitte,Tiefbau- und Entsorgungsdepartement (TED)
393,25.5.2005,2005/207,Hans Jörg Käppeli,SP,links,Heinz F. Steger,FDP,rechts,Tiefbau- und Entsorgungsdepartement (TED)
394,16.1.2008,2008/33,Marlène Butz,SP,links,Astrid Hirzel,CVP,mitte,Polizeidepartement (PD)
395,3.5.2006,2006/139,Marlène Butz,SP,links,Robert Schönbächler,CVP,mitte,Departement der Industriellen Betriebe (DIB)


In [119]:
#Auch diesen Dataframe speichern wir:
df.to_csv("Dataframe_bereinigt_linksuebergreifend.csv")

### Die Zahlen können wir schon einmal auswerten:

In [120]:
df_Anzahl = df["Partei 1"].value_counts().to_frame()
df_Anzahl["Partei 2"] = df["Partei 2"].value_counts()
df_Anzahl["Geschaefte"] = df_Anzahl["Partei 1"] + df_Anzahl["Partei 2"]
df_Anzahl = df_Anzahl.drop(columns=['Partei 1', 'Partei 2'])
df_Anzahl

,Geschaefte
SP,225.0
Grüne,95.0
FDP,95.0
AL,40.0
GLP,98.0
CVP,120.0
SVP,15.0
EVP,29.0
LdU,NaN


In [121]:
df["Departement"].value_counts()

Tiefbau- und Entsorgungsdepartement (TED)                                       108
Hochbaudepartement (HBD)                                                         40
Departement der Industriellen Betriebe (DIB)                                     31
Polizeidepartement (PD)                                                          31
Schul- und Sportdepartement (SSD)                                                30
Präsidialdepartement (PRD)                                                       29
Finanzdepartement (FD)                                                           25
Sozialdepartement (SD)                                                           23
Sicherheitsdepartement (SID)                                                     20
Gesundheits- und Umweltdepartement (GUD)                                         18
Hochbaudepartement (HBD), Schul- und Sportdepartement (SSD)                       2
Finanzdepartement (FD), Hochbaudepartement (HBD), Präsidialdepartement (PRD)

In [122]:
df[df["Block 1"] == "rechts"]["Departement"].value_counts()

Tiefbau- und Entsorgungsdepartement (TED)                                       14
Schul- und Sportdepartement (SSD)                                                6
Hochbaudepartement (HBD)                                                         6
Präsidialdepartement (PRD)                                                       4
Sicherheitsdepartement (SID)                                                     4
Departement der Industriellen Betriebe (DIB)                                     3
Gesundheits- und Umweltdepartement (GUD)                                         3
Finanzdepartement (FD)                                                           3
Sozialdepartement (SD)                                                           2
Finanzdepartement (FD), Hochbaudepartement (HBD), Präsidialdepartement (PRD)     1
Polizeidepartement (PD)                                                          1
Hochbaudepartement (HBD), Schul- und Sportdepartement (SSD)                      1
    

In [ ]:
df

### Jetzt zählen wir nach Legislaturen

bei diesen Positionen stimmt beim Datum etwas Grundsätzliches nicht, weshalb wir es mit einem annäherungsweisen Datum  aufgrund der Geschäftnummer ersetzen:

In [123]:
fehlerliste = ["29.01.20","08.01.20","18.12.19"]

for fehler in fehlerliste:
    fehlerposition = df[df["Datum"] == fehler].index.tolist()
    for zeile in fehlerposition:
        Jahr = df.loc[zeile, "Geschäft"].split("/")[0]
        Ersatzdatum = "31.12." + Jahr
        df.loc[zeile, "Datum"] = df.loc[zeile, "Datum"].replace(fehler, Ersatzdatum)

In [138]:
df["Datum"] = pd.to_datetime(df['Datum'], format='%d.%m.%Y')
df = df.set_index("Datum")
df.head(5)

,Geschäft,Einreichender 1,Partei 1,Block 1,Einreichender 2,Partei 2,Block 2,Departement
Datum,,,,,,,,
2011-09-28,2011/362,Alecs Recher,AL,links,Maleica Landolt,GLP,mitte,Finanzdepartement (FD)
2004-10-20,2004/541,Niklaus Scherr,AL,links,Kurt Maeder,CVP,mitte,Hochbaudepartement (HBD)
2013-05-16,2005/550,Mario Mariani,CVP,mitte,Muriel Herzig,Grüne,links,Tiefbau- und Entsorgungsdepartement (TED)
2016-01-06,2016/16,Christina Schiller,AL,links,Mario Babini,Parteilos,mitte,Polizeidepartement (PD)
2004-12-01,2004/637,Corine Mauch,SP,links,Albert Leiser,FDP,rechts,Finanzdepartement (FD)


In [139]:
df_1990 = df["1990-05-15":"1994-05-14"]
df_1994 = df["1994-05-15":"1998-05-14"]
df_1998 = df["1998-05-15":"2002-05-14"]
df_2002 = df["2002-05-15":"2006-05-14"]
df_2006 = df["2006-05-15":"2010-05-14"]
df_2010 = df["2010-05-15":"2014-05-14"]
df_2014 = df["2014-05-15":"2018-05-14"]
df_2018 = df["2018-05-15":"2022-05-14"]

print("1990-94: "+str(len(df_1990)))
print("1994-98: "+str(len(df_1994)))
print("1998-02: "+str(len(df_1998)))
print("2002-06: "+str(len(df_2002)))
print("2006-10: "+str(len(df_2006)))
print("2010-14: "+str(len(df_2010)))
print("2014-18: "+str(len(df_2014)))
print("2018-22: "+str(len(df_2018)))

1990-94: 0
1994-98: 4
1998-02: 19
2002-06: 64
2006-10: 71
2010-14: 72
2014-18: 73
2018-22: 57
